In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def extract_layer_output(model, layer_name, data):
    int_layer_op = model.get_layer(name=layer_name).output
    int_layer_model = Model(inputs=model.input, outputs=int_layer_op)
    int_layer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return int_layer_model.predict(data)

In [3]:
def generate_df(arr, label):
    df = pd.DataFrame(arr)
    df['label'] = label
    return df

In [4]:
base_path = 'combined/'
image_gen = ImageDataGenerator(rescale=1./255.)

train_flow = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(224, 224),
    batch_size=64,
    shuffle=False,
    class_mode='binary'
)

test_flow = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(224, 224),
    batch_size=1,
    shuffle = False,
    class_mode='binary'
)

Found 29452 images belonging to 2 classes.
Found 10030 images belonging to 2 classes.


In [5]:
y_test = test_flow.classes

In [6]:
custom_model = load_model('custom_model.h5')

In [8]:
int_output_custom = extract_layer_output(custom_model, 'global_average_pooling2d_1', train_flow)
int_output_custom.shape

(29452, 512)

In [9]:
int_output_custom = generate_df(int_output_custom, train_flow.classes)
int_output_custom.head()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,label
0,-0.114193,0.180711,0.349467,0.567677,0.343554,-0.476792,0.503344,0.482925,-0.462621,0.136693,...,-0.398593,-0.001034,-0.335969,0.856552,-0.158477,-0.349845,0.199385,0.331419,0.050204,0
1,-0.295818,-0.279723,0.898410,-0.093873,-0.068544,-0.155903,0.001103,0.873979,0.099372,-0.306599,...,0.279864,-0.112146,0.707446,-0.148069,-0.164078,-0.317907,-0.473865,0.127556,0.026406,0
2,-0.599102,-0.153469,-0.285542,-0.010906,0.186447,-0.410660,-0.044124,-0.357805,-0.036382,0.133819,...,-0.052587,-0.433729,-0.286950,0.128199,0.177232,0.488748,0.036320,0.184413,-0.457558,0
3,0.312241,-0.437902,-0.324559,-0.456228,-0.112724,-0.277492,-0.379832,-0.395910,0.120134,-0.095040,...,0.095117,-0.343859,0.284889,0.559849,-0.421915,0.129088,-0.042575,0.401339,-0.280539,0
4,-0.381651,-0.100388,0.066809,0.350780,-0.235437,-0.621862,-0.613681,-0.439923,-0.327604,-0.411732,...,0.190301,-0.434703,0.744278,0.140287,-0.414406,0.078920,-0.179400,-0.175216,-0.024499,0


In [10]:
int_output_custom.tail()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,label
29447,-0.990078,0.888346,0.590134,-0.686856,-0.798732,0.287124,-1.185333,0.042246,-0.512177,-0.120408,...,-0.104369,0.487388,0.699560,-0.471918,-1.068517,-0.542831,0.055369,0.524554,-1.383674,1
29448,0.259373,-0.352450,-0.219322,0.462698,0.058914,0.061175,-0.463058,-0.053835,0.065417,-0.442504,...,-0.501837,-0.365228,-0.233268,-0.228874,-0.271860,-0.520255,0.173958,0.310012,0.027164,1
29449,-0.330630,-0.924355,-0.493855,-0.078049,0.043347,0.166365,-0.860485,-0.325437,0.080329,0.854992,...,0.675910,-0.686102,0.036879,-0.310453,-0.279561,0.295570,0.132674,0.033526,-0.862672,1
29450,-0.444044,-0.392390,0.457789,0.652233,0.198594,-0.603673,-0.457933,-0.699784,-0.924572,0.838302,...,-1.190179,-0.362032,-0.516711,-0.306343,-1.192781,-1.159344,-0.131762,0.114807,-1.158191,1
29451,-0.626980,-0.035764,-0.198722,-0.682857,-0.132083,-0.394077,-0.872742,-0.789441,-0.276834,0.223408,...,-0.511272,-0.195264,-0.393153,-0.570841,-0.336735,-0.256344,-0.103844,0.823526,-0.584108,1


In [11]:
int_output_custom.to_csv('custom_model_rep.csv', index=False)

In [12]:
y_pred_custom = custom_model.predict(test_flow)

In [13]:
print("ROC AUC Score:", metrics.roc_auc_score(y_test, y_pred_custom))
print("AP Score:", metrics.average_precision_score(y_test, y_pred_custom))
print()
print(metrics.classification_report(y_test, y_pred_custom > 0.5))

ROC AUC Score: 0.9727627413358916
AP Score: 0.9743021069063329

              precision    recall  f1-score   support

           0       0.82      0.96      0.89      4650
           1       0.96      0.82      0.88      5380

    accuracy                           0.88     10030
   macro avg       0.89      0.89      0.88     10030
weighted avg       0.90      0.88      0.88     10030

